# Multiple Labeling Functions

During a search process, more than one labeling function can process each data slice. Let's go through an example using this dataset of retail transactions.

In [ ]:
import composeml as cp

columns = [
    'customer_id',
    'transaction_time',
    'amount',
    'brand',
    'product_id',
]

df = cp.demos.load_transactions()[columns]
df.head()

We will start by defining multiple labeling functions that calculate three target variables about the customers. The labeling functions will calculate the total amount they spent, their favorite brand, and the number of unique items they purchased for each transaction period.

In [ ]:
def total_spent(ds):
    """The total amount spent by a customer."""
    return ds.amount.sum()

def favorite_brand(ds):
    """The most frequnetly purchased brand."""
    return ds.brand.value_counts().idxmax()

def unique_products(ds):
    """The number of unique products purchased by a customer."""
    return ds.product_id.nunique()

Now, we can create the label maker while passing in a list that contains our labeling functions.

In [ ]:
lm = cp.LabelMaker(
    target_entity='customer_id',
    time_index='transaction_time',
    labeling_function=[
        total_spent,
        favorite_brand,
        unique_products,
    ],
    window_size='1h',
)

Running the search will return all target variables in the label times.

In [ ]:
lt = lm.search(
    df.sort_values('transaction_time'),
    num_examples_per_instance=3,
    verbose=False,
)

lt.head()

We will need to first select an individual target before applying transforms, plotting targets, or making labeling descriptions.

In [ ]:
lt.select('favorite_brand').head()